In [1]:

! pip install -U accelerate --quiet
! pip install -U transformers --quiet
! pip install -U datasets --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00


In [2]:
import accelerate
import transformers
import datasets

transformers.__version__, accelerate.__version__, datasets.__version__

('4.32.1', '0.22.0', '2.14.4')

In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

'''
datasets['train'] = datasets['train'].shuffle(seed=42).select(range(10000))
datasets['test'] = datasets['test'].shuffle(seed=42).select(range(100))
datasets['validation'] = datasets['validation'].shuffle(seed=42).select(range(100))
'''

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

"\ndatasets['train'] = datasets['train'].shuffle(seed=42).select(range(10000))\ndatasets['test'] = datasets['test'].shuffle(seed=42).select(range(100))\ndatasets['validation'] = datasets['validation'].shuffle(seed=42).select(range(100))\n"

In [5]:
base_model = "distilgpt2"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)


def tokenize_function(examples):
    return tokenizer(examples["text"])

In [6]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
#print(type(tokenized_datasets['train']), len(tokenized_datasets['train']))
#[print(tokenizer.decode(tokenized_datasets['train'][x]["input_ids"])) for x in range(5)]
#[print(len(tokenized_datasets['train'][x]["input_ids"]), tokenizer.decode(tokenized_datasets['train'][x]["input_ids"])) for x in range(10)]

In [8]:
# block_size = tokenizer.model_max_length
block_size = 128

In [9]:
def group_texts(examples):
    # Concatenate all texts.
    #print(examples.keys())
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [11]:
print(lm_datasets.keys())
print(len(lm_datasets['train']), len(lm_datasets['test']), len(lm_datasets['validation']))
print(lm_datasets['train'][1].keys())

dict_keys(['test', 'train', 'validation'])
18666 2209 1931
dict_keys(['input_ids', 'attention_mask', 'labels'])


In [12]:
#[print(tokenizer.decode(lm_datasets['train'][x]["input_ids"])) for x in range(5)]
#[print(len(lm_datasets['train'][x]["input_ids"]), tokenizer.decode(lm_datasets['train'][x]["input_ids"])) for x in range(10)]

In [36]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(base_model)

In [16]:
from transformers import Trainer, TrainingArguments

In [17]:
model_name = base_model.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs = 5,
    push_to_hub=False,
)

In [18]:
print(len(lm_datasets["train"]), len(lm_datasets["validation"]))

18666 1931


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.752700,3.664708
2,3.627900,3.644466
3,3.561800,3.634937
4,3.527500,3.631099
5,3.494200,3.632489


TrainOutput(global_step=11670, training_loss=3.6114759587383514, metrics={'train_runtime': 2639.588, 'train_samples_per_second': 35.358, 'train_steps_per_second': 4.421, 'total_flos': 3048353216593920.0, 'train_loss': 3.6114759587383514, 'epoch': 5.0})

In [52]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity (PPL): {math.exp(eval_results['eval_loss']):.2f}")

Perplexity (PPL): 37.81


In [50]:
from google.colab import drive
drive.mount('/content/drive')
trainer.save_model('/content/drive/My Drive/gpt_distil_ft')

Mounted at /content/drive


In [57]:
input_text = tokenizer('life is', return_tensors='pt')

actual_model = AutoModelForCausalLM.from_pretrained(base_model)
ft_model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/gpt_distil_ft')

output_ftm = ft_model.generate(**input_text, max_new_tokens=30)
print(tokenizer.decode(output_ftm[0], skip_special_tokens=True))

output_am = actual_model.generate(**input_text, max_new_tokens=30)
print(tokenizer.decode(output_am[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


life is a fictionalized version of the story, and the series is based on the story of a fictionalized version of the story. The series was first broadcast
life is a great way to get your hands on a new game.


















